### Friction

The dimensionless friction coeffcient $c_f$ relates bed stress $\tau$ [Pa] to current speed $u$ [m/s] at a specified elevation $z$ [m]. Friction velocity $u^{*}$ [m/s] is the square root of kinematic stress $\tau / \rho$ [m^2 s^-2], where $\rho$ [kg/m^3] is water density.

$\tau$ = $c_f \rho u^2 = \rho u^{*2}$ (1)

The law of the wall relates current speed to elevation as

$u = (u^{*}/\kappa) \text{log}(z/zo)$ (2)

Rearranging (2) to solve for $u^*$ and using (1) shows that

$c_f(zo) = [ \kappa / \text{log}(z/zo) ]^2 $ 

Manning's $n$ is a depth-dependent roughness coefficient:

$c_f(n) = [gn^2/h^{1/3}]^{1/2}$

Solving for $n$ in terms of $zo$:

$n = [h^{1/3}/g]^{1/2} [\kappa/\text{log}(z/zo)]^2$

Solving for $zo$ in terms of $n$:

$zo = \text{exp}[ -\kappa/(gn^2/h^{1/3}) + \text{log}(z) ]$

In [15]:
# check these formulae
import numpy as np

h = 10.
g = 9.81
vk = 0.41
z = 1.
zo = 0.002

cd_zo = (vk/np.log(z/zo))**2
print('cd_zo = ',cd_zo)

n = np.sqrt(h**(1./3.)/g) * (vk/np.log(z/zo))**2
print('n from zo = ',n)

cd_n  = np.sqrt( g*n**2/h**(1./3.) )
print('cd_n from n = ',cd_n)

zor = np.exp( np.log(z)-vk*( g*n**2 / h**(1./3.))**(-1./4.) )

print('zo from n: ',zor)

cd_zo =  0.00435251443532
n from zo =  0.00203972858939
cd_n from n =  0.00435251443532
zo from n:  0.002
